<a href="https://colab.research.google.com/github/StarMindz/Cool-nerds-location-identifier/blob/main/ITU_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing packages

!pip install transformers
!!pip install --upgrade --no-cache-dir gdown
!pip install torch

#Downloading the model files using gdown
!gdown --no-check-certificate   --folder https://drive.google.com/drive/u/1/folders/15d_hqZKPR2Z8OIe4oIzFQpWc3xWeyl_j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Retrieving folder list
Processing file 1-DjZVcd_P8PkjQo3eQWCeJeqmKIw-ntP config.json
Processing file 1-6MVs8ugLcwjHsNhWk7GrvY2ijp1Wdfp pytorch_model.bin
Processing file 1-BCRyuF7yqXt0z3TwbeExesU1nEBn1j2 special_tokens_map.json
Processing file 1-SarWEZSGWN-62BymeVDnlnaymjRp3Vg tokenizer_config.json
Processing file 1-EPzqsb6gRDT5m7Q_5pVKv7u6neJjxQE tokenizer.json
Processing file 1-XW1Y9RwNOSTbYEtUJ4rq3_i3xe0QNRc training_args.bin
Processing file 1-3pvDlYqLHqENgaTnzZmRHvF-F_9SVLc vocab.txt
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1-DjZVcd_P8PkjQo3eQWCeJeqmKIw-ntP
To: /content/un-ner.model/config.json
100% 1.11k/1.11k [00:00<00:00, 4.23MB/s]
Downloading...
From: https://drive.google.co

In [ ]:
import json, re, torch
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification

label_list = ['CONT','CTRY','STAT','CNTY','CITY','DIST','ISL','NBHD','ST','NPOI','HPOI','OTHR','O']

def predict_loc(text):

    """
        Predicts location in texts and
        returns their mentioned location
    """

    tokenizer = AutoTokenizer.from_pretrained('./un-ner.model/')

    tokens = tokenizer(text)
    torch.tensor(tokens['input_ids']).unsqueeze(0).size()

    model = AutoModelForTokenClassification.from_pretrained('./un-ner.model/', num_labels=len(label_list))
    predictions = model.forward(input_ids=torch.tensor(tokens['input_ids']).unsqueeze(0), attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))
    predictions = torch.argmax(predictions.logits.squeeze(), axis=1)
    predictions = [label_list[i] for i in predictions]

    words = tokenizer.batch_decode(tokens['input_ids'])
    out_labeled = {x:y for x,y in zip(words, predictions)}

    return out_labeled


def load_input_tweets(input_path):

    """
      function that reads the inputed tweets
    """

    tweets = {}
    for line in open(input_path, encoding='utf-8').read().splitlines(): 
        tweet = json.loads(line)
        tweets[tweet["tweet_id"]] = tweet["text"]
    return tweets


def model_output(tweets, path):

    """
       writes the model output to a json file
    """
    with open(path, 'w') as file:
        
        for i in range(len(tweets)):
            output = {}
            output["tweet_id"] = list(tweets.keys())[i]
            text = tweets[list(tweets.keys())[i]]
            prediction = predict_loc(text)
            locations = []
            for i in range(len(prediction)):
                if list(prediction.values())[i] != 'O':
                    word = list(prediction.keys())[i]
                    if word not in text.lower():
                        continue
                    start, end = re.search(word, text.lower()).span()
                    locations.append({"text":text[start:end], "start_offset": start, "end_offset": end})
            output["location_mentions"] = locations
            file.write(json.dumps(output) + "\n")


def predict(input_path, output_path):
  
    """
        make the lmr prediction
    """
    tweets = load_input_tweets(input_path)

    model_output(tweets, output_path)

    print("Model finished predictions!!!")


In [ ]:
input_file = 'input.jsonl'
output_file = 'output.jsonl'

predict(input_file, output_file)

Model finished predictions!!!
